In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [3]:
train = pd.read_csv("/Users/igyuseog/Desktop/open data/train.csv")
train = train.drop(train[train["familysize"] > 50].index)
test = pd.read_csv("/Users/igyuseog/Desktop/open data/test_x.csv")

drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}

train_y = train["voted"]
train_x = train.drop(drop_list + ["voted"], axis = 1)
test_x = test.drop(drop_list, axis = 1)

#타입 변경
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
#원핫인코딩
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
#numpy
train_y = 2 - train_y.to_numpy() #0이 투표한다. 1이 투표 안한다 
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()


# 정규화
train_x[:,:20] = (train_x[:,:20] -3.) / 2.
test_x[:,:20] = (test_x[:,:20] -3.) / 2.

train_x[:,20] = (train_x[:,20] -5.) / 5.
test_x[:,20] = (test_x[:,20] -5.) / 5.

train_x[:,21:31] = (train_x[:, 21:31] - 3.5) / 3.5
test_x[:,21:31] = (test_x[:, 21:31] - 3.5) / 3.5

#tensor
train_y = torch.tensor(train_y, dtype = torch.float32)
train_x = torch.tensor(train_x, dtype = torch.float32)
test_x = torch.tensor(test_x, dtype = torch.float32)
train_len, test_len = len(train_x), len(test_x)

#Dataloder
train_loader = DataLoader(TensorDataset(train_x,train_y), shuffle = True, drop_last = True, batch_size = 100)
test_loader =  DataLoader(TensorDataset(test_x, torch.zeros(len(test_x))), shuffle = False, drop_last = False)


In [14]:
train_x.size[0]

TypeError: 'builtin_function_or_method' object is not subscriptable

In [10]:
class DNN(torch.nn.Module):
    def __init__(self,input_size = 91):
        super(DNN,self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size,100),
            nn.ReLU(),
            
            nn.Linear(100,50),
            nn.ReLU(),
            
            nn.Linear(50, 1),
        )
                 
    def forward(self,x):
        #y_ =  x.view(x.size(0),-1)
        y_ = self.mlp(y_)
        return y_    

In [11]:
model = DNN()

criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor([1.20665]))
optimizer = optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 4e-2)

In [13]:
    
epochs = 10
step = 0

for idx, (xx,yy) in enumerate(train_loader):
    optimizer.zero_grad()
    pred = model(xx).squeeze()
    loss = criterion(pred, yy)
    loss.backward()
    optimizer.step()
    model.eval()
    if step % 100 == 0:
        print(step, loss.item())

    with torch.no_grad():
        test = []
        for idx, (xx,yy) in enumerate(test_loader):
            test.append(model(xx).squeeze())
    step += 1          

0 0.6601625680923462
100 0.6789780259132385
200 0.6740669012069702
300 0.5754489302635193
400 0.5625357031822205


In [118]:
test = np.array(test)
test = np.where(test > 1.5, 1, 2)

In [130]:
temp = pd.DataFrame(test)
temp[0].value_co

0        2
1        2
2        2
3        1
4        2
        ..
11378    2
11379    2
11380    2
11381    2
11382    2
Name: 0, Length: 11383, dtype: int64

In [ ]:
submit = 